# 1. Setup

Create the dataset directory and download the required dataset from Kaggle

In [ ]:
!mkdir datasets
%cd datasets
!mkdir traffic_sign_dataset
%cd traffic_sign_dataset
!kaggle datasets list -s gtsrb-german-traffic-sign
!kaggle datasets download meowmeowmeowmeowmeow/gtsrb-german-traffic-sign  

Unzip the Dataset to extract Training and Testing data.

In [ ]:
!tar -xf dogbreedidfromcomp.zip

Imports

In [ ]:
import os

import pandas as pd
import numpy as np

from PIL import Image
from sklearn.model_selection import  train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D

Constants

In [2]:
DATASET_PATH = "datasets/traffic_sign_dataset/{}"
TEST_PATH = DATASET_PATH.format("Test")
TRAIN_PATH = DATASET_PATH.format("Train")

ALL_LABELS = ['Speed limit (20km/h)','Speed limit (30km/h)','Speed limit (50km/h)','Speed limit (60km/h)',
              'Speed limit (70km/h)','Speed limit (80km/h)','End of speed limit (80km/h)','Speed limit (100km/h)',
              'Speed limit (120km/h)','No passing','No passing for vechiles over 3.5 metric tons',
              'Right-of-way at the next intersection','Priority road','Yield','Stop','No vechiles',
              'Vechiles over 3.5 metric tons prohibited','No entry','General caution','Dangerous curve to the left',
              'Dangerous curve to the right','Double curve','Bumpy road','Slippery road','Road narrows on the right',
              'Road work','Traffic signals','Pedestrians','Children crossing','Bicycles crossing','Beware of ice/snow',
              'Wild animals crossing','End of all speed and passing limits','Turn right ahead','Turn left ahead',
              'Ahead only','Go straight or right','Go straight or left','Keep right','Keep left','Roundabout mandatory',
              'End of no passing','End of no passing by vechiles over 3.5 metric']

# 2. Pre-Processing

Resize all images to (50,50)

In [ ]:
images = []
label_id = []

for i in range(43):
    labels = TRAIN_PATH + f"{i}"
    image_path = os.listdir(labels)
    for x in image_path:
        img = Image.open(labels + '/' + x)
        img = img.resize((50,50))
        img = np.array(img)
        images.append(img)
        label_id.append(i)
        
images = np.array(images)
label_id = np.array(label_id)

Normalise the pixel values

In [ ]:
images = images/255

Here we are finding out the count per class i.e. total data in each class using value_counts() function.

In [ ]:
label_counts = pd.DataFrame(label_id).value_counts()
label_counts.head()

# 3. Modelling

Train-Test split of data on 80%-20% ratio.

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(images, label_id , test_size = 0.2, random_state = 42)

Converting the classes column into categorical using to_categorical() function.

In [ ]:
y_train_cat = to_categorical(y_train)
y_val_cat = to_categorical(y_val)

Defining the model architecture. In this we will define all the layers with their input shape kernel size, activation, etc.

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (3,3), input_shape = x_train.shape[1:], activation = 'relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(43, activation = 'softmax'))

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Fit the model

In [ ]:
model.fit(x_train, y_train, epochs = 10, batch_size = 128, validation_data = (x_val, y_val), verbose = 2)

# 4. Analysis

Plot the Accuracy of the model per epoch to analyse how the model is learning with each epoch.

In [ ]:
evaluation = pd.DataFrame(model.history.history)
evaluation[['accuracy', 'val_accuracy']].plot()
evaluation[['loss', 'val_loss']].plot()

We will use predict function to make predictions using this model also we are finding out the accuracy on the test set.

In [ ]:
def scaling(test_images, test_path):
    images = []

    image_path = test_images
    
    for x in image_path:
        img = Image.open(test_path + '/' + x)
        img = img.resize((50,50))
        img = np.array(img)
        images.append(img)

    images = np.array(images)
    images = images/255

    return images

In [ ]:
test_images = scaling(sorted(os.listdir(TEST_PATH)), TEST_PATH)

In [ ]:
test = pd.read_csv(DATASET_PATH.format('Test.csv'))
y_test = test['ClassId'].values
y_pred = model.predict_classes(test_images)

In [ ]:
img = Image.open(TEST_PATH + '/00001.png')
print("Original label : ",ALL_LABELS[y_test[1]])
print("Predicted label : ",ALL_LABELS[y_pred[1]])